This notebook contains the code for the meta-analysis of healthy lung data for ACE2, TMPRSS2, and CTSL. It contains the pseudo-bulk analysis for the complex model with interaction terms that was run on the patient-level data.

In [1]:
import scanpy as sc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rcParams
from matplotlib import colors
from matplotlib import patches
import seaborn as sns
import batchglm
import diffxpy.api as de
import patsy as pat
from statsmodels.stats.multitest import multipletests
import logging, warnings
import statsmodels.api as sm

/home/icb/malte.luecken/anaconda3/envs/cov19_py/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()
de.__version__

logging.getLogger("tensorflow").setLevel(logging.ERROR)
logging.getLogger("batchglm").setLevel(logging.INFO)
logging.getLogger("diffxpy").setLevel(logging.INFO)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 35)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="tensorflow")

scanpy==1.4.5.1 anndata==0.7.1 umap==0.3.10 numpy==1.18.1 scipy==1.4.1 pandas==1.0.1 scikit-learn==0.23.1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


'v0.7.3'

In [3]:
#User inputs
folder = '/storage/groups/ml01/workspace/malte.luecken/2020_cov19_study'

adata_diffxpy = '/storage/groups/ml01/workspace/malte.luecken/2020_cov19_study/COVID19_lung_atlas_revision_v3.h5ad'

output_folder = 'diffxpy_out/'

de_output_base = 'COVID19_lung_atlas_revision_v3_lung_cov19_poissonglm_smoking_pseudo_nUMIoffset_testInts'

# Read the data

In [4]:
adata = sc.read(adata_diffxpy)

In [5]:
adata

AnnData object with n_obs × n_vars = 1320896 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new'

In [6]:
adata.obs.age = adata.obs.age.astype(float)

In [7]:
adata.obs.dtypes

age                         float64
anatomical_region          category
donor                      category
last_author/PI             category
lung_vs_nasal              category
notes                      category
original_celltype_ann      category
sample                     category
sex                        category
smoking                    category
total_counts                float64
smoked_boolean             category
last_author_sample_name    category
ann_level_1                category
ann_level_2                category
ann_level_3                category
ann_level_4                category
ann_level_5                category
ann_highest_res               int64
ann_new                        bool
dtype: object

In [8]:
adata.obs['dataset'] = adata.obs['last_author/PI']

In [9]:
adata.obs.dataset.value_counts()

Regev/Rajagopal            322998
Meyer_b                    117535
Kaminski                    95303
Spence                      78401
Barbry/Leroy                76981
Krasnow/Quake               60993
Meyer                       57020
Rawlins                     53409
Regev                       43527
Misharin/Budinger           41266
Eils/Conrad/Kreuter         39778
Seibold                     36248
Whitsett/Xu_10X             34185
Koenigshoff                 33119
Misharin                    28329
Xavier/Regev                25552
Spira/Campbell              24455
Lafyatis/Rojas              24220
Kropski/Banovich_vand       23285
Schultze                    22641
Schiller                    20776
Nawijn                      18197
Teichmann                   12971
Kropski/Banovich_dnar        8359
Shalek                       7603
Linnarsson                   4640
Whitsett/Xu_dropSeq          3267
Mazzilli/Campbell/Beane      2207
Schultze/Falk                1965
Beane         

# Filter the data

Keep only datsets with:
- more than 1 donor
- non-fetal
- lung

In [10]:
# Remove fetal datasets
dats_to_remove = set(['Rawlins', 'Spence', 'Linnarsson'])

In [11]:
dat = adata.obs.groupby(['donor']).agg({'sex':'first', 'age':'first', 'dataset':'first'})

# Single donor filter
don_tab = dat['dataset'].value_counts()
dats_to_remove.update(set(don_tab.index[don_tab == 1]))

In [12]:
dats_to_remove = list(dats_to_remove)
dats_to_remove

['Spence', 'Linnarsson', 'Misharin', 'Schultze/Falk', 'Rawlins']

In [13]:
adata = adata[~adata.obs.dataset.isin(dats_to_remove)].copy()

In [14]:
adata.obs.lung_vs_nasal.value_counts()

lung     1096604
nasal      57548
Name: lung_vs_nasal, dtype: int64

In [15]:
# Filter for only lung data
adata = adata[adata.obs.lung_vs_nasal.isin(['lung']),].copy()

In [16]:
adata

AnnData object with n_obs × n_vars = 1096604 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset'

In [17]:
adata.obs['sample'].nunique()
adata.obs['donor'].nunique()
adata.obs['dataset'].nunique()

309

185

24

# Binarize smoking status

In [18]:
adata.obs.smoking.value_counts()

never              575345
current            173020
nan                111184
former              53487
non-smoker          42367
never-smoker        33119
active              29396
smoked              28197
current/former      23573
heavy               15663
current/former?      5162
light                3316
Former               1385
Current               610
Never Smoker          426
Current Smoker        354
Name: smoking, dtype: int64

In [19]:
adata.obs['smoking_status'] = adata.obs.smoked_boolean
#adata.obs['smoking_status'] = [True if stat in ['current', 'smoked', 'active', 'former', 'heavy', 'light'] else False if stat in ['never', 'nonsmoker'] else "nan" for stat in adata.obs.smoking]

In [20]:
adata.obs.smoking_status.value_counts()

False    651257
True     334163
nan      111184
Name: smoking_status, dtype: int64

## Filter out data w/o smoking status

In [21]:
pd.crosstab(adata.obs.dataset, adata.obs.smoking)

smoking,Current,Current Smoker,Former,Never Smoker,active,current,current/former,current/former?,former,heavy,light,nan,never,never-smoker,non-smoker,smoked
dataset,,,,,,,,,,,,,,,,
Barbry/Leroy,0,0,0,0,0,0,0,0,16423,0,0,0,0,0,42367,0
Beane,0,0,0,0,0,305,0,0,581,0,0,0,0,0,0,0
Beane/Campbell,0,354,0,426,0,0,0,0,0,0,0,0,0,0,0,0
Eils/Conrad/Kreuter,0,0,0,0,0,11185,0,0,0,0,0,0,28593,0,0,0
Kaminski,0,0,0,0,0,0,23573,0,0,0,0,3850,67880,0,0,0
Koenigshoff,0,0,0,0,0,0,0,0,0,0,0,0,0,33119,0,0
Krasnow/Quake,0,0,0,0,0,0,0,0,7524,0,0,0,53469,0,0,0
Kropski/Banovich_dnar,0,0,0,0,0,0,0,0,0,0,0,0,2647,0,0,5712
Kropski/Banovich_vand,0,0,0,0,0,0,0,0,0,0,0,0,800,0,0,22485


In [22]:
adata = adata[~adata.obs.smoking_status.isin(['nan']),].copy()
adata

AnnData object with n_obs × n_vars = 985420 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset', 'smoking_status'

In [23]:
adata.obs.dataset.value_counts()
adata.obs['sample'].nunique()
adata.obs['donor'].nunique()

Regev/Rajagopal            322998
Meyer_b                    117535
Kaminski                    91453
Krasnow/Quake               60993
Barbry/Leroy                58790
Meyer                       57020
Misharin/Budinger           41266
Eils/Conrad/Kreuter         39778
Seibold                     36248
Whitsett/Xu_10X             34185
Koenigshoff                 33119
Lafyatis/Rojas              24220
Kropski/Banovich_vand       23285
Teichmann                   12971
Nawijn                      11110
Kropski/Banovich_dnar        8359
Schiller                     5162
Whitsett/Xu_dropSeq          3267
Mazzilli/Campbell/Beane      1995
Beane                         886
Beane/Campbell                780
Name: dataset, dtype: int64

286

164

# Check the data

In [24]:
np.mean(adata.X.astype(int) != adata.X)

0.0

In [25]:
for dat in adata.obs.dataset.unique():
    val = np.mean(adata[adata.obs.dataset.isin([dat]),:].X.astype(int) != adata[adata.obs.dataset.isin([dat]),:].X)
    if val != 0:
        print(f'dataset= {dat}; value= {val}')
        adata[adata.obs.dataset.isin([dat]),:].X[:20,:20].A

All counts are (or nearly are) integers

In [26]:
adata.obs.age.value_counts()
adata.obs.sex.value_counts()

57.00    81833
66.00    69371
42.00    66596
59.00    59955
18.00    59849
46.00    56291
64.00    46130
35.00    45869
0.25     29908
67.50    28201
20.00    25362
51.00    24766
30.00    24095
29.00    19248
0.00     19065
32.00    19009
3.00     18387
42.50    16906
41.00    14943
58.00    14086
57.50    13840
23.00    13657
68.00    11852
65.00    11404
56.00    11024
75.00    10941
45.00    10939
55.00    10878
47.00    10278
49.00    10198
21.00     9430
27.00     8693
72.50     7983
22.00     7980
63.00     7600
31.00     7085
33.00     6906
44.00     6681
52.50     6326
38.00     5629
62.00     4679
24.00     4433
62.50     4073
26.00     4066
61.00     3922
67.00     3916
50.00     3831
32.50     3772
80.00     3261
10.00     2694
40.00     2647
17.00     2552
79.00     2550
54.00     2507
76.00     2447
36.00     2218
48.00     1153
25.00      823
70.00      168
69.00      130
87.00      112
43.00       79
34.00       76
78.00       65
74.00       52
Name: age, dtype: int64

female    550340
male      435080
Name: sex, dtype: int64

# Fit models and perform DE

In [27]:
cluster_key = 'ann_level_2'
clust_tbl = adata.obs[cluster_key].value_counts()
clusters = clust_tbl.index[clust_tbl > 1000]
ct_to_rm = clusters[[ct.startswith('1') for ct in clusters]]
clusters = clusters.drop(ct_to_rm.tolist()).tolist()
clusters

['Myeloid',
 'Airway epithelium',
 'Alveolar epithelium',
 'Lymphoid',
 'Fibroblast lineage',
 'Blood vessels',
 'Submucosal Gland',
 'Smooth Muscle',
 'Lymphatics',
 'Mesothelium']

Calculate DE genes per cluster.

In [28]:
adata

AnnData object with n_obs × n_vars = 985420 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset', 'smoking_status'

# Generate pseudobulks

In [29]:
for gene in adata.var_names:
    adata.obs[gene] = adata[:,gene].X.A

In [30]:
dat_pseudo = adata.obs.groupby(['donor', 'ann_level_2']).agg({'ACE2':'mean', 'TMPRSS2':'mean', 'CTSL':'mean', 'total_counts':'mean', 'age':'first', 'smoking_status':'first', 'sex':'first', 'dataset':'first'}).dropna().reset_index(level=[0,1])
adata_pseudo = sc.AnnData(dat_pseudo[['ACE2', 'TMPRSS2', 'CTSL']], obs=dat_pseudo.drop(columns=['ACE2', 'TMPRSS2', 'CTSL']))

Transforming to str index.


In [31]:
adata_pseudo.obs.head()

,donor,ann_level_2,total_counts,age,smoking_status,sex,dataset
0,1A1DFBng,1_Endothelial,6307.156863,47.0,True,female,Eils/Conrad/Kreuter
1,1A1DFBng,Airway epithelium,4948.705263,47.0,True,female,Eils/Conrad/Kreuter
2,1A1DFBng,Alveolar epithelium,4268.209154,47.0,True,female,Eils/Conrad/Kreuter
3,1A1DFBng,Fibroblast lineage,5888.260870,47.0,True,female,Eils/Conrad/Kreuter
4,1A1DFBng,Lymphatics,5503.117647,47.0,True,female,Eils/Conrad/Kreuter


In [32]:
adata_pseudo.obs['total_counts_scaled'] = adata_pseudo.obs['total_counts']/adata_pseudo.obs['total_counts'].mean()

In [33]:
# Get interquartile range for ages to test
adata_pseudo.obs.groupby(['donor']).agg({'age':'first'}).age.quantile([0.25,0.5,0.75])

0.25    30.75
0.50    49.50
0.75    62.50
Name: age, dtype: float64

In [34]:
formula = "1 + sex + age + sex:age + smoking_status + sex:smoking_status + age:smoking_status + dataset"
tested_coef = ["sex[T.male]", "age", "smoking_status[T.True]"]
dmat = de.utils.design_matrix(
    data=adata,
    formula="~" + formula,
    as_numeric=["age"],
    return_type="patsy"
)

to_test = dict()
to_test['age'] = [31,62]
to_test['sex[T.male]'] = [0,1]
to_test['smoking_status[T.True]'] = [0,1]

dmat[1]

['Intercept',
 'sex[T.male]',
 'smoking_status[T.True]',
 'dataset[T.Beane]',
 'dataset[T.Beane/Campbell]',
 'dataset[T.Eils/Conrad/Kreuter]',
 'dataset[T.Kaminski]',
 'dataset[T.Koenigshoff]',
 'dataset[T.Krasnow/Quake]',
 'dataset[T.Kropski/Banovich_dnar]',
 'dataset[T.Kropski/Banovich_vand]',
 'dataset[T.Lafyatis/Rojas]',
 'dataset[T.Mazzilli/Campbell/Beane]',
 'dataset[T.Meyer]',
 'dataset[T.Meyer_b]',
 'dataset[T.Misharin/Budinger]',
 'dataset[T.Nawijn]',
 'dataset[T.Regev/Rajagopal]',
 'dataset[T.Schiller]',
 'dataset[T.Seibold]',
 'dataset[T.Teichmann]',
 'dataset[T.Whitsett/Xu_10X]',
 'dataset[T.Whitsett/Xu_dropSeq]',
 'sex[T.male]:smoking_status[T.True]',
 'age',
 'sex[T.male]:age',
 'age:smoking_status[T.True]']

### Function definition to test effect sizes at particular covariate values

In [35]:
def calc_effects(dmat, cov_mat, params, effect, coefs):
    from patsy.design_info import DesignMatrix
    from diffxpy.api.stats import wald_test_chisq
    
    dmat_cond = isinstance(dmat, tuple) and isinstance(dmat[0], DesignMatrix)

    if not dmat_cond:
        raise ValueError("`dmat` should be a patsy output Design Matrix.")

    effect_list = ['sex[T.male]', 'age', 'smoking_status[T.True]']
    if not effect in effect_list:
        raise ValueError(f'{effect} is not one of: '
                         f'{effect_list}')

    if not isinstance(coefs, dict):
        raise TypeError('`coefs` should contain a dictionary of coefficients '
                        'where the effects should be evaluated.')
    
    ## Note: this is only correct when 3 covariates are tested in combinations
    #if np.sum([coef in coefs for coef in effect_list]) < 2:
    #    raise ValueError('The `coefs` dict must contain values for the two '
    #                     'coefficient not tested in:'
    #                     f'{effect_list}')
        
    if 'smoking_status[T.True]' in coefs and coefs['smoking_status[T.True]'] not in [0,1]:
        raise ValueError('Smoking status should be encoded as 0 or 1.')

    if 'sex[T.male]' in coefs and coefs['sex[T.male]'] not in [0,1]:
        raise ValueError('Sex should be encoded as 0 or 1.')

    if 'age' in coefs and not (isinstance(coefs['age'], float) or isinstance(coefs['age'], int)):
        raise ValueError('Age should be a numerical value.')

    coef_list = []

    for term in dmat[1]:
        if effect not in term:
            coef_list.append(0)
        elif term == effect:
            coef_list.append(1)
        else:
            t_list = term.split(':')
            t_list.remove(effect)
            coef_list.append(coefs[t_list[0]])

    C = np.array(coef_list)
    val = np.matmul(C,np.array(params))
    stderr = np.sqrt(np.matmul(np.matmul(C.T,cov_mat),C))
    pval = wald_test_chisq(np.array([val]).reshape(1,1), np.array([stderr**2]).reshape(1,1,1))[0]
    return (val, stderr, pval)

## Poisson GLM

In [36]:
# Poisson GLM loop
de_results_lvl2_glm = dict()

# Test over clusters
for clust in clusters:
    adata_pseudo_tmp = adata_pseudo[adata_pseudo.obs[cluster_key] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_pseudo_tmp.obs['smoking_status'].value_counts())
    print(adata_pseudo_tmp.obs['sex'].value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_pseudo_tmp, min_cells=4)
    if adata_pseudo_tmp.n_vars == 0:
        print('No genes expressed in more than 10 cells!')
        continue
    if len(adata_pseudo_tmp.obs.smoking_status.value_counts())==1:
        print(f'{clust} only has 1 type of smoker/nonsmoker sample.')
        continue
        
    print(f'Testing {adata_pseudo_tmp.n_vars} genes...')
    print(f'Testing in {adata_pseudo_tmp.n_obs} donors...')   
    print("")

    # List to store results
    de_results_list = []        

    # Set up design matrix
    dmat = de.utils.design_matrix(
        data=adata_pseudo_tmp, #[idx_train],
        formula="~" + formula,
        as_numeric=["age"],
        return_type="patsy"
    )
    
    # Test if model is full rank
    if np.linalg.matrix_rank(np.asarray(dmat[0])) < np.min(dmat[0].shape):
        print(f'Cannot test {clust} as design matrix is not full rank.')
        continue
    
    for i, gene in enumerate(adata_pseudo_tmp.var_names):
        # Specify model
        pois_model = sm.GLM(
            endog=adata_pseudo_tmp.X[:, i], #[idx_train, :], 
            exog=dmat[0],
            offset=np.log(adata_pseudo_tmp.obs['total_counts_scaled'].values),
            family=sm.families.Poisson()
        )

        # Fit the model
        pois_results = pois_model.fit()

        # Get the covariance matrix
        cov_mat = pois_results.cov_params()

        # Test over coefs
        for coef in tested_coef:
            iter_coefs = tested_coef.copy()
            iter_coefs.remove(coef)
            
            for c1 in to_test[iter_coefs[0]]:
                for c2 in to_test[iter_coefs[1]]:
                    coef_vals = {iter_coefs[0]:c1, iter_coefs[1]:c2}
                    val, stderr, pval = calc_effects(
                        dmat = dmat,
                        cov_mat = cov_mat,
                        params = pois_results.params,
                        effect = coef,
                        coefs = coef_vals)
                    
                    case = '_'.join([iter_coefs[0]+':'+str(c1), iter_coefs[1]+':'+str(c2)])
                    case = case.replace('smoking_status[T.True]:0','NS').replace('smoking_status[T.True]:1','S')
                    case = case.replace('sex[T.male]:0','F').replace('sex[T.male]:1','M')
                    case = case.replace('age:31','31yr').replace('age:62','62yr')
                    case = case.replace('_',' ')
                    
                    # Output the results nicely
                    de_results_temp = pd.DataFrame({
                        "gene": gene,
                        "cell_identity": clust,
                        "covariate": coef,
                        "eval_at": case,
                        "coef": val,
                        "coef_sd": stderr,                 
                        "pval": pval
                    }, index= [clust+"_"+gene+"_"+coef])
                    
                    de_results_list.append(de_results_temp)

    de_results = pd.concat(de_results_list)
    de_results['adj_pvals'] = multipletests(de_results['pval'].tolist(), method='fdr_bh')[1]
    
    # Store the results
    de_results_lvl2_glm[clust] = de_results
    
# Join the dataframes:
full_res_lvl2_glm = pd.concat([de_results_lvl2_glm[i] for i in de_results_lvl2_glm.keys()], ignore_index=True)

In cluster Myeloid:
False    66
True     55
Name: smoking_status, dtype: int64
male      64
female    57
Name: sex, dtype: int64
Testing 3 genes...
Testing in 121 donors...

In cluster Airway epithelium:
False    90
True     71
Name: smoking_status, dtype: int64
male      85
female    76
Name: sex, dtype: int64
Testing 3 genes...
Testing in 161 donors...

In cluster Alveolar epithelium:
False    67
True     39
Name: smoking_status, dtype: int64
male      55
female    51
Name: sex, dtype: int64
Testing 3 genes...
Testing in 106 donors...

In cluster Lymphoid:
False    72
True     62
Name: smoking_status, dtype: int64
male      70
female    64
Name: sex, dtype: int64
Testing 3 genes...
Testing in 134 donors...

In cluster Fibroblast lineage:
False    54
True     40
Name: smoking_status, dtype: int64
male      50
female    44
Name: sex, dtype: int64
Testing 3 genes...
Testing in 94 donors...

In cluster Blood vessels:
False    48
True     31
Name: smoking_status, dtype: int64
male      45

## Inspect some results

In [37]:
de_results_lvl2_glm.keys()

dict_keys(['Myeloid', 'Airway epithelium', 'Alveolar epithelium', 'Lymphoid', 'Fibroblast lineage', 'Blood vessels', 'Submucosal Gland', 'Smooth Muscle', 'Lymphatics', 'Mesothelium'])

In [38]:
full_res_lvl2_glm

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],31yr NS,0.360704,10.722035,0.973163,0.979942
1,ACE2,Myeloid,sex[T.male],31yr S,-1.516361,25.708715,0.952966,0.979942
2,ACE2,Myeloid,sex[T.male],62yr NS,0.536208,11.653562,0.963300,0.979942
3,ACE2,Myeloid,sex[T.male],62yr S,-1.340856,22.520670,0.952523,0.979942
4,ACE2,Myeloid,age,F NS,-0.023072,0.306370,0.939970,0.979942
5,ACE2,Myeloid,age,F S,0.058640,0.837871,0.944204,0.979942
6,ACE2,Myeloid,age,M NS,-0.017411,0.468073,0.970328,0.979942
7,ACE2,Myeloid,age,M S,0.064302,0.920595,0.944315,0.979942
8,ACE2,Myeloid,smoking_status[T.True],F 31yr,-1.256606,21.668032,0.953754,0.979942
9,ACE2,Myeloid,smoking_status[T.True],F 62yr,1.276476,12.905233,0.921209,0.979942


In [39]:
full_res_lvl2_glm.loc[full_res_lvl2_glm['gene'] == 'ACE2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],31yr NS,0.360704,10.722035,0.973163,0.979942
1,ACE2,Myeloid,sex[T.male],31yr S,-1.516361,25.708715,0.952966,0.979942
2,ACE2,Myeloid,sex[T.male],62yr NS,0.536208,11.653562,0.963300,0.979942
3,ACE2,Myeloid,sex[T.male],62yr S,-1.340856,22.520670,0.952523,0.979942
4,ACE2,Myeloid,age,F NS,-0.023072,0.306370,0.939970,0.979942
5,ACE2,Myeloid,age,F S,0.058640,0.837871,0.944204,0.979942
6,ACE2,Myeloid,age,M NS,-0.017411,0.468073,0.970328,0.979942
7,ACE2,Myeloid,age,M S,0.064302,0.920595,0.944315,0.979942
8,ACE2,Myeloid,smoking_status[T.True],F 31yr,-1.256606,21.668032,0.953754,0.979942
9,ACE2,Myeloid,smoking_status[T.True],F 62yr,1.276476,12.905233,0.921209,0.979942


In [40]:
full_res_lvl2_glm.loc[full_res_lvl2_glm['gene'] == 'TMPRSS2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
12,TMPRSS2,Myeloid,sex[T.male],31yr NS,1.029464,2.779169,0.711069,0.979942
13,TMPRSS2,Myeloid,sex[T.male],31yr S,0.287078,3.952527,0.942099,0.979942
14,TMPRSS2,Myeloid,sex[T.male],62yr NS,1.044418,2.328614,0.653782,0.979942
15,TMPRSS2,Myeloid,sex[T.male],62yr S,0.302032,3.873930,0.937856,0.979942
16,TMPRSS2,Myeloid,age,F NS,0.012739,0.060758,0.833933,0.979942
17,TMPRSS2,Myeloid,age,F S,-0.018189,0.188118,0.922974,0.979942
18,TMPRSS2,Myeloid,age,M NS,0.013221,0.114626,0.908175,0.979942
19,TMPRSS2,Myeloid,age,M S,-0.017707,0.178052,0.920784,0.979942
20,TMPRSS2,Myeloid,smoking_status[T.True],F 31yr,0.564781,3.706406,0.878887,0.979942
21,TMPRSS2,Myeloid,smoking_status[T.True],F 62yr,-0.393970,3.570754,0.912146,0.979942


# Level 3 annotation

In [41]:
cluster_key = 'ann_level_3'
clust_tbl = adata.obs[cluster_key].value_counts()
clusters = clust_tbl.index[clust_tbl > 1000]
ct_to_rm = clusters[[ct.startswith('1') or ct.startswith('2') for ct in clusters]]
clusters = clusters.drop(ct_to_rm.tolist()).tolist()
clusters

['Macrophages',
 'Basal',
 'AT2',
 'T cell lineage',
 'Monocytes',
 'Multiciliated lineage',
 'Submucosal Secretory',
 'AT1',
 'Innate lymphoid cells',
 'Secretory',
 'Capillary',
 'Mast cells',
 'B cell lineage',
 'Fibroblasts',
 'Dendritic cells',
 'Venous',
 'Lymphatic EC',
 'Arterial',
 'Rare',
 'Airway smooth muscle']

In [42]:
adata_sub = adata[adata.obs.ann_level_3.isin(clusters),:]

adata_sub
adata_sub.obs.donor.nunique()
adata_sub.obs['sample'].nunique()

View of AnnData object with n_obs × n_vars = 795195 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset', 'smoking_status', 'ACE2', 'TMPRSS2', 'CTSL'

164

286

## Generate pseudobulk

In [43]:
for gene in adata_sub.var_names:
    adata_sub.obs[gene] = adata_sub[:,gene].X.A

Trying to set attribute `.obs` of view, copying.


In [44]:
dat_pseudo_sub = adata_sub.obs.groupby(['donor', 'ann_level_3']).agg({'ACE2':'mean', 'TMPRSS2':'mean', 'CTSL':'mean', 'total_counts':'mean', 'age':'first', 'smoking_status':'first', 'sex':'first', 'dataset':'first'}).dropna().reset_index(level=[0,1])
adata_pseudo_sub = sc.AnnData(dat_pseudo_sub[['ACE2', 'TMPRSS2', 'CTSL']], obs=dat_pseudo_sub.drop(columns=['ACE2', 'TMPRSS2', 'CTSL']))

Transforming to str index.


In [45]:
adata_pseudo_sub.obs.head()

,donor,ann_level_3,total_counts,age,smoking_status,sex,dataset
0,1A1DFBng,AT1,3709.913265,47.0,True,female,Eils/Conrad/Kreuter
1,1A1DFBng,AT2,4453.519898,47.0,True,female,Eils/Conrad/Kreuter
2,1A1DFBng,Lymphatic EC,5503.117647,47.0,True,female,Eils/Conrad/Kreuter
3,1A1DFBng,Monocytes,3339.083948,47.0,True,female,Eils/Conrad/Kreuter
4,1A1DFBng,Multiciliated lineage,6000.121212,47.0,True,female,Eils/Conrad/Kreuter


In [46]:
adata_pseudo_sub.obs['total_counts_scaled'] = adata_pseudo_sub.obs['total_counts']/adata_pseudo_sub.obs['total_counts'].mean()

## Poisson GLM

First check if there are any datasets with only 1 sex or 1 smoking status, which would make the model overparameterized (not full rank).

In [47]:
np.any(adata_pseudo_tmp.obs.smoking_status.value_counts() == 1)

False

In [48]:
np.any(pd.crosstab(adata_pseudo_tmp.obs.smoking_status, adata_pseudo_tmp.obs.sex) == 1)

False

In [49]:
clusters

['Macrophages',
 'Basal',
 'AT2',
 'T cell lineage',
 'Monocytes',
 'Multiciliated lineage',
 'Submucosal Secretory',
 'AT1',
 'Innate lymphoid cells',
 'Secretory',
 'Capillary',
 'Mast cells',
 'B cell lineage',
 'Fibroblasts',
 'Dendritic cells',
 'Venous',
 'Lymphatic EC',
 'Arterial',
 'Rare',
 'Airway smooth muscle']

In [50]:
# Poisson GLM loop
de_results_lvl3_glm = dict()

# Test over clusters
for clust in clusters:
    adata_pseudo_tmp = adata_pseudo_sub[adata_pseudo_sub.obs[cluster_key] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_pseudo_tmp.obs['smoking_status'].value_counts())
    print(adata_pseudo_tmp.obs['sex'].value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_pseudo_tmp, min_cells=4)
    if adata_pseudo_tmp.n_vars == 0:
        print('No genes expressed in more than 10 cells!')
        continue
    if len(adata_pseudo_tmp.obs.smoking_status.value_counts())==1:
        print(f'{clust} only has 1 type of smoker/nonsmoker sample.')
        continue
    if np.any(adata_pseudo_tmp.obs.smoking_status.value_counts()==1):
        print(f'{clust} only has 1 smoker or 1 nonsmoker sample.')
        continue
    if np.any(adata_pseudo_tmp.obs.sex.value_counts()==1):
        print(f'{clust} only has 1 male or 1 female sample.')
        continue
    if np.any(pd.crosstab(adata_pseudo_tmp.obs.smoking_status, adata_pseudo_tmp.obs.sex) == 1):
        print('Want at least 2 in each smoking/sex category.')
        continue
        
    print(f'Testing {adata_pseudo_tmp.n_vars} genes...')
    print(f'Testing in {adata_pseudo_tmp.n_obs} donors...')
    print("")

    # List to store results
    de_results_list = []        

    # Set up design matrix
    dmat = de.utils.design_matrix(
        data=adata_pseudo_tmp,
        formula="~" + formula,
        as_numeric=["age"],
        return_type="patsy"
    )
    
    # Test if model is full rank
    if np.linalg.matrix_rank(np.asarray(dmat[0])) < np.min(dmat[0].shape):
        print(f'Cannot test {clust} as design matrix is not full rank.')
        continue
    
    for i, gene in enumerate(adata_pseudo_tmp.var_names):
        # Specify model
        pois_model = sm.GLM(
            endog=adata_pseudo_tmp.X[:, i],
            exog=dmat[0],
            offset=np.log(adata_pseudo_tmp.obs['total_counts_scaled'].values),
            family=sm.families.Poisson()
        )

        # Fit the model
        pois_results = pois_model.fit()

        # Get the covariance matrix
        cov_mat = pois_results.cov_params()

        # Test over coefs
        for coef in tested_coef:
            iter_coefs = tested_coef.copy()
            iter_coefs.remove(coef)
            
            for c1 in to_test[iter_coefs[0]]:
                for c2 in to_test[iter_coefs[1]]:
                    coef_vals = {iter_coefs[0]:c1, iter_coefs[1]:c2}
                    val, stderr, pval = calc_effects(
                        dmat = dmat,
                        cov_mat = cov_mat,
                        params = pois_results.params,
                        effect = coef,
                        coefs = coef_vals)
                    
                    case = '_'.join([iter_coefs[0]+':'+str(c1), iter_coefs[1]+':'+str(c2)])
                    case = case.replace('smoking_status[T.True]:0','NS').replace('smoking_status[T.True]:1','S')
                    case = case.replace('sex[T.male]:0','F').replace('sex[T.male]:1','M')
                    case = case.replace('age:31','31yr').replace('age:62','62yr')
                    case = case.replace('_',' ')
                    
                    # Output the results nicely
                    de_results_temp = pd.DataFrame({
                        "gene": gene,
                        "cell_identity": clust,
                        "covariate": coef,
                        "eval_at": case,
                        "coef": val,
                        "coef_sd": stderr,                 
                        "pval": pval
                    }, index= [clust+"_"+gene+"_"+coef])
                    
                    de_results_list.append(de_results_temp)        

    de_results = pd.concat(de_results_list)
    de_results['adj_pvals'] = multipletests(de_results['pval'].tolist(), method='fdr_bh')[1]
    
    # Store the results
    de_results_lvl3_glm[clust] = de_results
    
# Join the dataframes:
full_res_lvl3_glm = pd.concat([de_results_lvl3_glm[i] for i in de_results_lvl3_glm.keys()], ignore_index=True)

In cluster Macrophages:
False    49
True     40
Name: smoking_status, dtype: int64
male      52
female    37
Name: sex, dtype: int64
Testing 3 genes...
Testing in 89 donors...

In cluster Basal:
False    53
True     52
Name: smoking_status, dtype: int64
male      58
female    47
Name: sex, dtype: int64
Testing 3 genes...
Testing in 105 donors...

In cluster AT2:
False    66
True     38
Name: smoking_status, dtype: int64
male      53
female    51
Name: sex, dtype: int64
Testing 3 genes...
Testing in 104 donors...

In cluster T cell lineage:
False    51
True     46
Name: smoking_status, dtype: int64
male      53
female    44
Name: sex, dtype: int64
Testing 3 genes...
Testing in 97 donors...

In cluster Monocytes:
False    57
True     39
Name: smoking_status, dtype: int64
male      51
female    45
Name: sex, dtype: int64
Testing 3 genes...
Testing in 96 donors...

In cluster Multiciliated lineage:
False    88
True     69
Name: smoking_status, dtype: int64
male      83
female    74
Name: s

## Inspect some results

In [51]:
de_results_lvl3_glm.keys()

dict_keys(['Macrophages', 'Basal', 'AT2', 'T cell lineage', 'Monocytes', 'Multiciliated lineage', 'Submucosal Secretory', 'AT1', 'Innate lymphoid cells', 'Secretory', 'Capillary', 'Mast cells', 'B cell lineage', 'Fibroblasts', 'Dendritic cells', 'Venous', 'Lymphatic EC', 'Arterial', 'Rare', 'Airway smooth muscle'])

In [52]:
full_res_lvl3_glm

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],31yr NS,-0.297912,14.560428,0.983676,0.995306
1,ACE2,Macrophages,sex[T.male],31yr S,-1.574722,31.857020,0.960576,0.995306
2,ACE2,Macrophages,sex[T.male],62yr NS,-1.640307,23.518489,0.944396,0.995306
3,ACE2,Macrophages,sex[T.male],62yr S,-2.917117,24.878505,0.906658,0.995306
4,ACE2,Macrophages,age,F NS,0.005383,0.377046,0.988610,0.995306
...,...,...,...,...,...,...,...,...
619,CTSL,Airway smooth muscle,age,M S,-0.017950,0.071221,0.801017,0.965744
620,CTSL,Airway smooth muscle,smoking_status[T.True],F 31yr,2.959046,6.590653,0.653449,0.965744
621,CTSL,Airway smooth muscle,smoking_status[T.True],F 62yr,0.097870,2.278868,0.965744,0.965744
622,CTSL,Airway smooth muscle,smoking_status[T.True],M 31yr,2.170441,6.084910,0.721322,0.965744


In [53]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'ACE2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],31yr NS,-0.297912,14.560428,0.983676,0.995306
1,ACE2,Macrophages,sex[T.male],31yr S,-1.574722,31.857020,0.960576,0.995306
2,ACE2,Macrophages,sex[T.male],62yr NS,-1.640307,23.518489,0.944396,0.995306
3,ACE2,Macrophages,sex[T.male],62yr S,-2.917117,24.878505,0.906658,0.995306
4,ACE2,Macrophages,age,F NS,0.005383,0.377046,0.988610,0.995306
5,ACE2,Macrophages,age,F S,0.008195,1.011467,0.993535,0.995306
6,ACE2,Macrophages,age,M NS,-0.037920,0.705416,0.957129,0.995306
7,ACE2,Macrophages,age,M S,-0.035108,0.929224,0.969862,0.995306
8,ACE2,Macrophages,smoking_status[T.True],F 31yr,0.824854,33.871399,0.980571,0.995306
9,ACE2,Macrophages,smoking_status[T.True],F 62yr,0.912047,12.894138,0.943610,0.995306


In [54]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'TMPRSS2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
12,TMPRSS2,Macrophages,sex[T.male],31yr NS,-0.073281,4.509511,0.987035,0.995306
13,TMPRSS2,Macrophages,sex[T.male],31yr S,-0.028752,4.887415,0.995306,0.995306
14,TMPRSS2,Macrophages,sex[T.male],62yr NS,-0.512749,5.950561,0.931333,0.995306
15,TMPRSS2,Macrophages,sex[T.male],62yr S,-0.468221,3.943872,0.905496,0.995306
16,TMPRSS2,Macrophages,age,F NS,0.005882,0.111655,0.957986,0.995306
17,TMPRSS2,Macrophages,age,F S,0.005481,0.170912,0.974418,0.995306
18,TMPRSS2,Macrophages,age,M NS,-0.008294,0.173562,0.961885,0.995306
19,TMPRSS2,Macrophages,age,M S,-0.008696,0.166749,0.958411,0.995306
20,TMPRSS2,Macrophages,smoking_status[T.True],F 31yr,0.295882,5.274646,0.955266,0.995306
21,TMPRSS2,Macrophages,smoking_status[T.True],F 62yr,0.283439,4.320157,0.947690,0.995306


# Store results

In [55]:
#res_summary_lvl2.to_csv(folder+'/'+output_folder+de_output_base+'_lvl2_summary.csv')
full_res_lvl2_glm.to_csv(folder+'/'+output_folder+de_output_base+'_lvl2_full.csv')

In [56]:
#res_summary_lvl3.to_csv(folder+'/'+output_folder+de_output_base+'_lvl3_summary.csv')
full_res_lvl3_glm.to_csv(folder+'/'+output_folder+de_output_base+'_lvl3_full.csv')